In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from random import shuffle
from sklearn.model_selection import train_test_split
from sklearn import metrics,svm
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from skimage.measure import regionprops
from skimage.filters import threshold_otsu
from sklearn.preprocessing import MaxAbsScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def load_data(datadir, classes, img_size=100):
    training_data = []
    label = []
    for classe in range(len(classes)):
        path = os.path.join(datadir, classes[classe])
        shufled_list  = list(os.listdir(path))
        shuffle(shufled_list)
        for img in shufled_list:
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            img_array = cv2.resize(img_array, (img_size, img_size))
            unique = np.unique(img_array)
            if len(unique) == 1:
                continue
            training_data.append(img_array)
            label.append(classe)
    return training_data , label

In [ ]:
data , label = load_data('/content/drive/MyDrive/CV/geometric',['circle','square','star','triangle'])

In [ ]:
l_fire = np.zeros(len(d_dir))
l_dandelion = np.ones(len(dandelion_dir))
l_rose = 2*np.ones(len(rose_dir))
l_sunflower = 3*np.ones(len(sunflower_dir))
l_tulip = 4*np.ones(len(tulip_dir))
y = np.concatenate((l_daisy, l_dandelion, l_rose, l_sunflower, l_tulip))
y = to_categorical(y, 5)

In [ ]:
def get_contours_param(contour):
    contour_area = contour[0].filled_area
    contour_perimeter = contour[0].perimeter
    contour_convex_area = contour[0].convex_area
    diameter = contour[0].equivalent_diameter
    return contour_area , contour_perimeter, contour_convex_area, diameter

In [ ]:
def features_extraction(images):
    features_list = []
    for image in images:
        thresh = threshold_otsu(image)
        binary = np.array(image > thresh).astype(int)
        white_pixel = np.where(binary > 0)
        if len(white_pixel[0]) > 7000:
            binary = abs(1-binary) # ajuste de imagens negativas
        regions = regionprops(binary)
        contour_area , contour_perimeter, contour_convex_area, diameter = get_contours_param(regions)
        features_list.append([contour_area , contour_perimeter, contour_convex_area, diameter])
    norm =  MaxAbsScaler()
    norm.fit(features_list)
    norm_features = norm.transform(features_list)
    return norm_features

In [ ]:
features = features_extraction(data)

In [ ]:
def generate_svm_model(train_data,label_train_data,test_data):
    clf = svm.SVC(kernel='linear')
    clf.fit(train_data, label_train_data)
    predicted = clf.predict(test_data)
    return predicted
def generate_SGDC_model(train_data,label_train_data,test_data):
    clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=200)
    clf.fit(train_data, label_train_data)
    predicted = clf.predict(test_data)
    return predicted
def generate_naive_bayes_model(train_data,label_train_data,test_data):
    gnb = GaussianNB()
    gnb.fit(train_data, label_train_data)
    predicted = gnb.predict(test_data)
    return predicted
def generate_decision_tree_model(train_data,label_train_data,test_data):
    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(train_data, label_train_data)
    predicted = clf.predict(test_data)
    return predicted
def generate_random_forest_model(X_train, y_train,test_data):
    rfc = RandomForestClassifier(criterion= 'entropy', max_depth= 8, max_features='auto', n_estimators=200)
    rfc.fit(X_train,y_train)
    predicted = rfc.predict(test_data)
    return predicted
def generate_MLP_model(X_train, y_train,test_data):
    classifier = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=300,activation = 'relu',solver='adam',random_state=1)
    classifier.fit(X_train, y_train)
    predicted = classifier.predict(test_data)
    return predicted
def generate_knn_model(train_data,label_train_data,test_data):
    knn = KNeighborsClassifier()
    knn.fit(train_data,label_train_data)
    predicted = knn.predict(test_data)
    return predicted

In [ ]:
def gen_classifiers(train_data,label_train_data,test_data):
    return generate_knn_model(train_data,label_train_data,test_data),\
    generate_MLP_model(train_data,label_train_data,test_data),\
    generate_SGDC_model(train_data,label_train_data,test_data),\
    generate_svm_model(train_data,label_train_data,test_data),\
    generate_decision_tree_model(train_data,label_train_data,test_data),\
    generate_naive_bayes_model(train_data,label_train_data,test_data),\
    generate_random_forest_model(train_data,label_train_data,test_data),


In [ ]:
X_train,X_test,y_train,y_test = train_test_split(features,label,test_size=0.3)
results = gen_classifiers(X_train, y_train,X_test)

In [ ]:
display(results[0])

array([2, 1, 0, 3, 0, 1, 0, 0, 0, 3, 3, 3, 3, 2, 3, 1, 0, 2, 3, 2, 3, 0,
       0, 1, 2, 1, 3, 2, 3, 2, 0, 0, 3, 2, 3, 1, 2, 3, 0, 0, 2, 1, 3, 0,
       0, 1, 2, 0, 0, 2, 3, 2, 2, 2, 0, 3, 0, 0, 2, 0, 3, 0, 1, 1, 1, 2,
       1, 2, 3, 3, 3, 2, 0, 2, 0, 1, 3, 2, 0, 3, 0, 2, 3, 2, 2, 1, 0, 0,
       1, 0, 1, 2, 0, 2, 0, 1, 3, 2, 2, 0, 2, 3, 3, 2, 3, 0, 1, 0, 2, 2,
       3, 0, 1, 0, 3, 2, 2, 3, 1, 3, 2, 2, 3, 3, 1, 2, 3, 2, 3, 3, 1, 1,
       0, 2, 3, 1, 0, 0, 3, 2, 3, 0, 2, 3, 0, 2, 0, 2, 0, 0, 2, 0, 2, 1,
       3, 1, 3, 2, 3, 0, 1, 2, 0, 1, 3, 3, 3, 2, 0, 2, 2, 0, 3, 2, 3, 2,
       3, 2, 2, 1, 3, 0, 1, 3, 0, 0, 3, 0, 0, 2, 3, 1, 0, 1, 3, 1, 0, 3,
       3, 0, 3, 1, 3, 2, 0, 1, 2, 2, 3, 3, 1, 3, 3, 1, 2, 3, 3, 2, 2, 0,
       2, 2, 3, 1, 3, 0, 1, 0, 1, 3, 1, 3, 1, 1, 0, 2, 0, 1, 2, 3, 3, 2,
       2, 1, 3, 1, 1, 0, 0, 0, 3, 0, 3, 1, 0, 0, 1, 0, 2, 0, 1, 3, 1, 3,
       3, 3, 0, 2, 2, 3, 1, 0, 2, 3, 3, 2, 0, 1, 3, 0, 0, 2, 2, 1, 3, 2,
       2, 0, 2, 3, 3, 0, 2, 3, 3, 3, 3, 0, 1, 3, 1,

In [ ]:
models = ['KNN', 'MLP', 'SGDC', 'SVM', 'Árvore de Decisão', 'NB', 'RF']

for i in range(len(models)):
  print(models[i])
  acc = metrics.accuracy_score(y_test,results[i])
  recall = metrics.recall_score(y_test,results[i],average=None)
  precision = metrics.precision_score(y_test,results[i],average=None,zero_division=1)
  f1 = metrics.f1_score(y_test,results[i],average=None)
  print('Acurácia: ', acc)
  print('Recall: ', recall)
  print('Precisão: ', precision)
  print('F1-score: ', f1)
  print('\n')

KNN
Acurácia:  0.9708029197080292
Recall:  [0.94615385 0.91346154 1.         1.        ]
Precisão:  [0.95348837 0.93137255 1.         0.98224852]
F1-score:  [0.94980695 0.9223301  1.         0.99104478]


MLP
Acurácia:  0.9963503649635036
Recall:  [1.         0.98076923 1.         1.        ]
Precisão:  [0.98484848 1.         1.         1.        ]
F1-score:  [0.99236641 0.99029126 1.         1.        ]


SGDC
Acurácia:  0.6751824817518248
Recall:  [0.6        0.         1.         0.86746988]
Precisão:  [0.70909091 1.         0.83146067 0.55384615]
F1-score:  [0.65       0.         0.90797546 0.67605634]


SVM
Acurácia:  0.6313868613138686
Recall:  [0.47692308 0.         0.7972973  1.        ]
Precisão:  [0.71264368 1.         1.         0.48396501]
F1-score:  [0.57142857 0.         0.88721805 0.65225933]


Árvore de Decisão
Acurácia:  0.9288321167883211
Recall:  [0.93846154 0.79807692 0.97297297 0.96385542]
Precisão:  [0.91729323 0.88297872 0.97959184 0.91954023]
F1-score:  [0.92775